# Necessary Steps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MediaCloud

/content/drive/MyDrive/MediaCloud


In [ ]:
import os
pwd = os.getcwd()
pwd

'/content/drive/MyDrive/MediaCloud'

# Install Libraries

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentencepiece
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Hugging Face and Libraries

In [ ]:
pwd

'/content/drive/MyDrive/MediaCloud'

In [ ]:
from huggingface_hub import login

# Use your Hugging Face access token (get it from https://huggingface.co/settings/tokens)
login('ENTER TOKER HERE')

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
import pandas as pd
import os

# Model

In [ ]:
model = "meta-llama/Llama-3.2-1B-Instruct"
#model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    #max_length=4096,
    max_new_tokens=1024,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
#    truncation=True
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

<ipython-input-10-9410ce7de563>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


# Output - All at Once

In [ ]:
input_folder = 'Farmers_Protest/2.Parsed_YearWise_DataTEMP'

output_folder = 'Farmers_Protest/3.Parsed_YearWise_Data_with_Summaries'
os.makedirs(output_folder, exist_ok=True)


In [ ]:
template = """
              Write a long summary of the following text delimited by triple backticks, including all key points and supporting details.
              The summary should accurately reflect the main message and arguments presented in the original text.
              ```{text}```
              SUMMARY:
           """

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

<ipython-input-13-e769f4a73240>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
# Loop through all CSV files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):  # Process only CSV files
        # Construct full file path
        input_file = os.path.join(input_folder, file_name)

        # Load the DataFrame
        df = pd.read_csv(input_file)

        # Summarize the 'parsed_text' column
        df["Summary"] = df["parsed_text"].apply(lambda text: llm_chain.run(text))

        # Save the updated DataFrame to the output folder
        output_file = os.path.join(output_folder, file_name.replace('.csv', '_with_summaries.csv'))
        df.to_csv(output_file, index=False)

        # Print success message for each file
        print(f"Processed and saved: {output_file}")

print("All files processed and saved successfully!")

<ipython-input-14-0b0675ef74fe>:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  df["Summary"] = df["parsed_text"].apply(lambda text: llm_chain.run(text))
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed and saved: Farmers_Protest/3.Parsed_YearWise_Data_with_Summaries/Parsed_data_2024_with_summaries.csv
All files processed and saved successfully!


# Just Summaries

In [ ]:
pwd

'/content/drive/MyDrive/MediaCloud'

In [ ]:
import pandas as pd
import re
import os
import glob

In [ ]:

input_folder = 'Farmers_Protest/3.Parsed_YearWise_Data_with_Summaries'

output_folder = 'Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


In [ ]:
def extract_summary(text):
    # Regex to extract everything after 'SUMMARY:' (case insensitive)
    match = re.search(r'SUMMARY:\s*(.*)', text, re.IGNORECASE)
    return match.group(1) if match else ''


In [ ]:
csv_files = glob.glob(os.path.join(input_folder, '*.csv'))


In [ ]:
# Loop through each file
for file_path in csv_files:
    # Extract the file name without the folder path
    file_name = os.path.basename(file_path)
    output_file = os.path.join(output_folder, file_name)

    print(f"Processing file: {file_name}")

    # Load the CSV into a DataFrame
    df = pd.read_csv(file_path)

    # Check if the column 'parsed_summary' exists
    if 'Summary' in df.columns:
        # Apply the extract_summary function
        df['Summary'] = df['Summary'].apply(extract_summary)

        # Save the updated DataFrame to the output folder
        df.to_csv(output_file, index=False)
        print(f"Saved updated CSV to: {output_file}")
    else:
        print(f"Column 'Summary' not found in {file_name}. Skipping this file.")

print("Processing complete.")

Processing file: Parsed_data_2022_with_summaries.csv
Saved updated CSV to: Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries/Parsed_data_2022_with_summaries.csv
Processing file: Parsed_data_2021_with_summaries.csv
Saved updated CSV to: Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries/Parsed_data_2021_with_summaries.csv
Processing file: Parsed_data_2020_with_summaries.csv
Saved updated CSV to: Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries/Parsed_data_2020_with_summaries.csv
Processing file: Parsed_data_2024_with_summaries.csv
Saved updated CSV to: Farmers_Protest/4.UPDATED_Parsed_YearWise_Data_with_Summaries/Parsed_data_2024_with_summaries.csv
Processing complete.
